# Run this notebook second

This notebook will run the Fast Gradient Sign Method attack on a star-galaxy dataset, using a classifier trained in a different notebook. You will need to update some of the options (e.g., `data_dir`, possibly `ckpt_path` if you renamed it, etc.) below for your local situation. If you need a copy of the data, see the instructions at the root-level README.

The notebook will produce a set of HDF5 files containing images with the FGSM attack applied with names like `fgsm_0_XYZ.hdf5` for epsilon (attack "magnitude") 0.XYZ (float).

In [1]:
import sys
import logging
import time

In [2]:
sys.path.append("..")

In [3]:
from ptlib.dataloaders import StarGalaxyDataManager as DataManager
from ptlib.models import SimpleSGConvNet as Model
from ptlib.attackers import FGSMAttacker as Attacker
from ptlib.utils import get_logging_level
from ptlib.utils import log_function_args

In [6]:
git_hash=!git describe --abbrev=12 --dirty --always
git_hash = git_hash[0]
print(git_hash)

0d5702e19db6


In [7]:
ckpt_path="sg_ckpt.tar"
data_dir="/Users/perdue/Dropbox/Data/Workspace"
epsilons = [0.0, 0.01, 0.05]
log_freq=20
log_level="INFO"
short_test=False

Lots of useful information will go to the log file:

In [9]:
logfilename = 'log_attack_star_galaxy' + str(int(time.time())) + '.txt'
print(logfilename)

log_attack_star_galaxy1595417491.txt


In [10]:
logging.basicConfig(
    filename=logfilename, level=get_logging_level(log_level),
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
LOGGER = logging.getLogger(__name__)
LOGGER.info("Starting...")
log_function_args(vars())

Set up a data manager and a model:

In [11]:
data_manager = DataManager(data_dir=data_dir)
data_manager.make_means()
model = Model()

Set up the attacker:

In [12]:
attacker = Attacker(data_manager, model, ckpt_path, log_freq)
attacker.restore_model_and_optimizer()

Run the attack for each epsilon (FGSM scale factor):

In [13]:
for epsilon in epsilons:
    attacker.attack_for_single_epsilon(
        epsilon, short_test=short_test)